In [15]:
import skimage.io
import numpy as np
import os
import matplotlib.pyplot
from skimage.transform import resize
from skimage import filters
from sklearn import svm
from sklearn.svm import LinearSVC

In [16]:
def create_dataset(img_folder):
   
    img_data = []
    img_label = []
    IMG_WIDTH = 64
    IMG_HEIGHT = 128

    for dir1 in os.listdir(img_folder):
        for file in os.listdir(os.path.join(img_folder, dir1)):
       
            image_path = os.path.join(img_folder, dir1,  file)
            img = skimage.io.imread(image_path, as_gray=True)
            img = resize(img, (128, 64), anti_aliasing=False)
            img_data.append(img)

            if dir1 == 'pos':
                img_label.append(1)
            else:
                img_label.append(0)

    return np.array(img_data), np.array(img_label)


def get_data_from_dataset():
    train_img_data, train_img_label = create_dataset(r"F:\Courses Data\M.Phil\2nd semester\Pattern Recognition\Assignment 1\PETA\dataset\train")
    test_img_data, test_img_label = create_dataset(r"F:\Courses Data\M.Phil\2nd semester\Pattern Recognition\Assignment 1\PETA\dataset\test")
    return train_img_data, train_img_label, test_img_data, test_img_label

In [17]:
def cal_hist_bin_for_cell(grad_magnitude, grad_direction, row, col):
    bins = np.zeros(9)
    for i in range(8):
        for j in range(8):
            if (grad_direction[row + i, col + j] % 20 == 0):
                idx = int(grad_direction[row + i, col + j] // 20)
                if (idx == 9):
                    idx = 0
                bins[idx] += grad_magnitude[row + i, col + j]
            else:
                idx1 = int(grad_direction[row + i, col + j] // 20)
                mag1 = (abs((idx1 * 20) - grad_direction[row + i, col + j]) / 20) * grad_magnitude[row + i, col + j]
                if (idx1 + 1 == 9):
                    idx2 = 0
                    mag2 = ((180 - grad_direction[row + i, col + j]) / 20) * grad_magnitude[row + i, col + j]
                else:
                    idx2 = idx1 + 1
                    mag2 = (abs((idx2 * 20) - grad_direction[row + i, col + j]) / 20) * grad_magnitude[row + i, col + j]

                if (mag1 == mag2):
                    bins[idx1] += mag1
                    bins[idx2] += mag2
                elif (mag1 > mag2):
                    bins[idx2] += mag1
                    bins[idx1] += mag2
                else:
                    bins[idx1] += mag1
                    bins[idx2] += mag2
    return bins

In [18]:
def compute_all_cell_bins(grad_magnitude, grad_direction):
    bins_arr = np.zeros((0, 9))

    for i in range(0, 128, 8):
        for j in range(0, 64, 8):
            
            cell_bins = cal_hist_bin_for_cell(grad_magnitude, grad_direction, i, j)
            # stack all cell bins vertically
            bins_arr = np.vstack((bins_arr, cell_bins))
#             matplotlib.pyplot.bar(x=np.arange(9), height=cell_bins, align="center", width=0.8)
#             matplotlib.pyplot.show()
    return bins_arr

In [19]:
def rearrange_bins(bins_arr):
    cells = np.zeros((9, 16, 8))
    k = 0
    for i in range(16):
        for j in range(8):
            cells[:, i, j] = bins_arr[k, :]
            k += 1
    return cells

In [20]:
def perform_normalization(block_bins_arr):
    normalized_blocks = np.zeros((36, 15, 7))
    for i in range(15):
        for j in range(7):
            cell1 = block_bins_arr[:, i, j]
            cell2 = block_bins_arr[:, i, j + 1]
            cell3 = block_bins_arr[:, i + 1, j]
            cell4 = block_bins_arr[:, i + 1, j + 1]
            
            block = np.concatenate((cell1, cell2, cell3, cell4), axis=None)
            
            block_sum = np.sqrt(np.sum(np.square(block)))
            
            if (block_sum != 0): # necessay check to avoid NaN values placement because of division by zero
                normalized_blocks[:, i, j] = block / block_sum
            else:
                normalized_blocks[:, i, j] = block

    return normalized_blocks

In [21]:
def compute_vector_for_svc(data):
    img_vectors = np.zeros((data.shape[0], 3780))
    
    for i in range(data.shape[0]):
        img = data[i, :, :]

#         img = skimage.io.imread('lenna.png', as_gray=True)
#         skimage.io.imshow(img)

#         img = resize(img, (128, 64), anti_aliasing=True) # anti_alising means whether to apply smoothing or not
#         skimage.io.imshow(img)

        x_grad = filters.sobel_h(img)
        y_grad = filters.sobel_v(img)

        grad_magnitude = np.sqrt(np.square(x_grad) + np.square(y_grad))

        # taking mod with 180 to convert -ve angles to their equivalent co-terminal angles
        grad_direction = np.around(np.rad2deg(np.arctan2(y_grad, x_grad)) % 180, decimals=0)

        bins_arr = compute_all_cell_bins(grad_magnitude, grad_direction)

        block_bins_arr = rearrange_bins(bins_arr)

        normalized_blocks = perform_normalization(block_bins_arr)

        normalized_blocks = normalized_blocks.reshape(3780) 

#         matplotlib.pyplot.bar(x=np.arange(3780), height=normalized_blocks, align="center", width=0.8)
#         matplotlib.pyplot.show()

        img_vectors[i, :] = normalized_blocks

    return img_vectors

In [22]:
train_img_data, train_img_label, test_img_data, test_img_label = get_data_from_dataset()

training_data = compute_vector_for_svc(train_img_data)
testing_data = compute_vector_for_svc(test_img_data)

clf = LinearSVC(C=0.01)
clf.fit(training_data, train_img_label)
print("Score: ", clf.score(testing_data, test_img_label))

Score:  0.571078431372549
